# Lyft: Training with multi-mode confidence

![](http://www.l5kit.org/_images/av.jpg)
<cite>The image from L5Kit official document: <a href="http://www.l5kit.org/README.html">http://www.l5kit.org/README.html</a></cite>

Continued from the previous kernel:
 - [Lyft: Comprehensive guide to start competition](https://www.kaggle.com/corochann/lyft-comprehensive-guide-to-start-competition)
 - [Lyft: Deep into the l5kit library](https://www.kaggle.com/corochann/lyft-deep-into-the-l5kit-library)

In this kernel, I will run **pytorch CNN model training**. Especially, followings are new items to try:
 - Predict **multi-mode with confidence**: As written in [evaluation metric](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/overview/evaluation) page, we can predict **3 modes** of motion trajectory.
 - Training loss with **competition evaluation metric**
 - Use Training abstraction library **`pytorch-ignite` and `pytorch-pfn-extras`**.


[Update 2020/9/6]<br/>
Published prediction kernel: [Lyft: Prediction with multi-mode confidence](https://www.kaggle.com/corochann/lyft-prediction-with-multi-mode-confidence)<br/>
Try yourself how good score you can get using only single model without ensemble! :)

# Environment setup

 - Please add [pestipeti/lyft-l5kit-unofficial-fix](https://www.kaggle.com/pestipeti/lyft-l5kit-unofficial-fix) as utility script.
    - Official utility script "[philculliton/kaggle-l5kit](https://www.kaggle.com/mathurinache/kaggle-l5kit)" does not work with pytorch GPU.
 - Please add [lyft-config-files](https://www.kaggle.com/jpbremer/lyft-config-files) as dataset
 
See previous kernel [Lyft: Comprehensive guide to start competition](https://www.kaggle.com/corochann/lyft-comprehensive-guide-to-start-competition) for details.

In [1]:
# https://github.com/pfnet/pytorch-pfn-extras/releases/tag/v0.3.1
# !pip install pytorch-pfn-extras==0.6.1

In [2]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
# pd.set_option('max_columns', 50)


/tmp/ipykernel_3619098/525924121.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import zarr

import l5kit
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='jshtml')
print("l5kit version:", l5kit.__version__)

l5kit version: 1.5.0


In [4]:
import torch
from pathlib import Path

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
import pytorch_pfn_extras.training.extensions as E

## Function

To define loss function to calculate competition evaluation metric **in batch**.<br/>
It works with **pytorch tensor, so it is differentiable** and can be used for training Neural Network.

In [5]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape
    # print(f"pred.shape : {pred.shape}")
    assert gt.shape == (batch_size, future_len, num_coords), f"wrong shape for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Model

pytorch model definition. Here model outputs both **multi-mode trajectory prediction & confidence of each trajectory**.

In [6]:
# --- Model utils ---
import torch
from torchvision.models import resnet18
from torch import nn
from typing import Dict



In [7]:
# --- Utils ---
import yaml


def save_yaml(filepath, content, width=120):
    with open(filepath, 'w') as f:
        yaml.dump(content, f, width=width)


def load_yaml(filepath):
    with open(filepath, 'r') as f:
        content = yaml.safe_load(f)
    return content


class DotDict(dict):
    """dot.notation access to dictionary attributes

    Refer: https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary/23689767#23689767
    """  # NOQA

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    

## Configs

In [8]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': 'resnet50',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'render_ego_history':True,
        'step_time':0.1
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,

        'set_origin_to_bottom': True,
        'disable_traffic_light_faces':False
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 12,
        'shuffle': True,
        'num_workers': 4
    },

    'valid_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'max_num_steps': 10000,
        'checkpoint_every_n_steps': 5000,

        # 'eval_every_n_steps': -1
    }
}





In [9]:
flags_dict = {
    "debug": True,
    # --- Data configs ---
    "l5kit_data_folder": "/home/zhuhe/kaggle/input/lyft-motion-prediction-autonomous-vehicles",
    # --- Model configs ---
    "pred_mode": "multi",
    # --- Training configs ---
    "device": "cuda:0",
    "out_dir": "results/multi_train",
    "epoch": 2,
    "snapshot_freq": 50,
}

# Main script

Now finished defining all the util codes. Let's start writing main script to train the model!

## Loading data

Here we will only use the first dataset from the sample set. (sample.zarr data is used for visualization, please use train.zarr / validate.zarr / test.zarr for actual model training/validation/prediction.)<br/>
We're building a `LocalDataManager` object. This will resolve relative paths from the config using the `L5KIT_DATA_FOLDER` env variable we have just set.

In [10]:
flags = DotDict(flags_dict)
out_dir = Path(flags.out_dir)
os.makedirs(str(out_dir), exist_ok=True)
print(f"flags: {flags_dict}")
save_yaml(out_dir / 'flags.yaml', flags_dict)
save_yaml(out_dir / 'cfg.yaml', cfg)
debug = flags.debug



flags: {'debug': True, 'l5kit_data_folder': '/home/zhuhe/kaggle/input/lyft-motion-prediction-autonomous-vehicles', 'pred_mode': 'multi', 'device': 'cuda:0', 'out_dir': 'results/multi_train', 'epoch': 2, 'snapshot_freq': 50}


In [11]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = flags.l5kit_data_folder
dm = LocalDataManager(None)

print("Load dataset...")
train_cfg = cfg["train_data_loader"]
valid_cfg = cfg["valid_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)
train_path = "scenes/train.zarr" if debug else train_cfg["key"]
train_zarr = ChunkedDataset(dm.require(train_path)).open()
print("train_zarr", type(train_zarr))
train_ego_dataset = EgoDataset(cfg, train_zarr, rasterizer)
train_agent_dataset = AgentDataset(cfg, train_zarr, rasterizer)

# valid_path = "scenes/sample.zarr" if debug else valid_cfg["key"]
# valid_zarr = ChunkedDataset(dm.require(valid_path)).open()
# print("valid_zarr", type(train_zarr))
# valid_agent_dataset = AgentDataset(cfg, valid_zarr, rasterizer)
# valid_dataset = TransformDataset(valid_agent_dataset, transform)


Load dataset...
train_zarr <class 'l5kit.data.zarr_dataset.ChunkedDataset'>


In [12]:
def get_distance(centroid1, centroid2):
    return np.sqrt(np.square(centroid1[0]-centroid2[0]) + np.square(centroid1[1]-centroid2[1]))
# def change_all_list_to_ndarray(dic_in):

def transform_one_scene_dataset(scene_ego_dataset, scene_agent_dataset, debug: False):
    def add_element_into(ele_dict, ele):
        temp_x = []
        temp_x.extend(ele["curr_speed"].flatten())
        temp_x.extend(ele["history_positions"].flatten())
        temp_x.extend(ele["history_yaws"].flatten())
        temp_x.extend(ele["history_availabilities"].flatten())
        ele_dict["x"].append(temp_x)
        if len(ele_dict["index"])== 0:
            cur_index = 0
        else:
            cur_index = ele_dict["index"][-1] + 1
        for i in range(0,cur_index):
            ele_dict["edge_attr"].append(get_distance(ele["centroid"], ele_dict["centroid"][i])) # 双向图
            ele_dict["edge_index"][0].append(i)
            ele_dict["edge_index"][1].append(cur_index)
            ele_dict["edge_attr"].append(get_distance(ele["centroid"], ele_dict["centroid"][i]))
            ele_dict["edge_index"][1].append(i)
            ele_dict["edge_index"][0].append(cur_index)
        ele_dict["target_positions"].append(ele["target_positions"])
        ele_dict["target_availabilities"].append(ele["target_availabilities"])
        ele_dict["centroid"].append(ele["centroid"])  # 这个元素是为了建图方便
        ele_dict["index"].append(cur_index)  # 这个元素是为了建图方便

    return_np = [] # 这个的长度应该是247或者248的样子，并且这个就是frame_index

    for ele in scene_ego_dataset:
        ele_dict = {}
        for name in ["x","edge_index","edge_attr","centroid","index","target_positions","target_availabilities"]:
            if name == "edge_index":
                ele_dict[name] = [[],[]]
            else:
                ele_dict[name] = []
        add_element_into(ele_dict,ele)
        return_np.append(ele_dict)
    if debug:
        print(f"len(return_np) : {len(return_np)}")
    for ele in scene_agent_dataset:
        curr_frame_index = ele["frame_index"]
        if debug:
            print(ele["frame_index"])
        cur_dict = return_np[curr_frame_index]
        add_element_into(cur_dict, ele)

    for ele in return_np:
        for key_ in ele.keys():
            ele[key_] = np.array(ele[key_])

        ele["x"] = torch.tensor(ele["x"])
        ele["edge_index"] = torch.tensor(ele["edge_index"],dtype=torch.long)
        ele["edge_attr"] = torch.tensor(ele["edge_attr"])
        ele["target_positions"] = torch.tensor(ele["target_positions"])
        ele["target_availabilities"] = torch.tensor(ele["target_availabilities"])

    return return_np

## Prepare model & optimizer

In [13]:

import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False, whether_dropout= False):
        # TODO: Implement this function that initializes self.convs,
        # self.bns, and self.softmax.

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = None

        # A list of 1D batch normalization layers
        self.bns = None

        # The log softmax layer
        self.softmax = None
        self.dropout = dropout

        self.convs = torch.nn.ModuleList()

        for i in range(num_layers - 1):
            self.convs.append(GCNConv(input_dim, hidden_dim))
            input_dim = hidden_dim

        self.convs.append(GCNConv(input_dim, output_dim))
        self.whether_dropout = whether_dropout

        self.softmax=torch.nn.LogSoftmax()
        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds



    # def forward(self, x, edge_index,edge_attr):
    #     # x = x.to(device) # 在模型to_divice很慢？
    #     edge_attr = edge_attr.to(device)
    #     edge_index = edge_index.to(device)
    #     out = None
    #     for layer in range(len(self.convs)-1):  #layer：层数
    #         x=self.convs[layer](x,edge_index,edge_attr).to(torch.float)   #叠GCNConv
    #         # x= x.to(torch.float)# 这个是因为他这的输出搞成了float64,导致大家数据形式不兼容
    #         x=F.relu(x)  #叠relu,这个不会导致数据被转化成float,
    #         if self.whether_dropout is True:
    #             x=F.dropout(x,self.dropout,self.training)  #叠dropout。这个self.dropout看下文是概率。
    #     #最后一层
    #     out=self.convs[-1](x,edge_index,edge_attr)  #GCNVonv
    #     if not self.return_embeds:
    #         out=self.softmax(out)
    def forward(self, batch_data):
        out = None
        x = batch_data.x
        for layer in range(len(self.convs)-1):  #layer：层数
            x=self.convs[layer](x,batch_data.edge_index,batch_data.edge_attr).to(torch.float)   #叠GCNConv

            # x= x.to(torch.float)# 这个是因为他这的输出搞成了float64,导致大家数据形式不兼容
            x=F.relu(x)  #叠relu,这个不会导致数据被转化成float,
            if self.whether_dropout is True:
                x=F.dropout(x,self.dropout,self.training)  #叠dropout。这个self.dropout看下文是概率。
        #最后一层

        out=self.convs[-1](x,batch_data.edge_index,batch_data.edge_attr)  #GCNVonv
        if not self.return_embeds:
            out=self.softmax(out)

        return out




In [14]:
args = {
    'device' : 'cuda' if torch.cuda.is_available() else 'cpu',
    # 'device' = 'cpu'
    'num_layers': 2,
    'hidden_dim': 256,
    'dropout': 0,
    'lr': 0.001,
    'epochs': 30,
    'train_scene_start': 10,
    'train_scene_end': 30,
    'test_scene_start': 10,
    'test_scene_end': 30,
}

In [15]:
#dict_keys(['frame_index', 'image', 'target_positions', 'target_yaws', 'target_velocities', 'target_availabilities', 'history_positions', 'history_yaws', 'history_velocities', 'history_availabilities', 'world_to_image', 'raster_from_agent', 'raster_from_world', 'agent_from_world', 'world_from_agent', 'centroid', 'yaw', 'extent', 'history_extents', 'future_extents', 'curr_speed', 'scene_index', 'host_id', 'timestamp', 'track_id'])

In [16]:
scene_ego_dataset = train_ego_dataset.get_scene_dataset(60)
scene_agent_dataset = train_agent_dataset.get_scene_dataset(60)
frame_dic_array = transform_one_scene_dataset(scene_ego_dataset,scene_agent_dataset,debug=False)

/home/zhuhe/anaconda3/envs/pytorch/lib/python3.9/site-packages/l5kit/data/zarr_dataset.py:213: RuntimeWarning:

zarr dataset path should end with .zarr (for now). Open will fail for this dataset!

/home/zhuhe/anaconda3/envs/pytorch/lib/python3.9/site-packages/l5kit/dataset/agent.py:139: RuntimeWarning:

you're running with a custom agents_mask



In [17]:
from tqdm import tqdm,trange

input_dim = len(frame_dic_array[0]["x"][0])
model_gcn = GCN(input_dim=input_dim, hidden_dim=args["hidden_dim"], output_dim = cfg["model_params"]["future_num_frames"] * 2,num_layers=args["num_layers"],dropout=args["dropout"],return_embeds = True, whether_dropout=False)

model_gcn = model_gcn.to(device=args['device'])

print(model_gcn.eval())
epoch = flags.epoch

optimizer = torch.optim.Adam(model_gcn.parameters(), lr=args["lr"])

GCN(
  (convs): ModuleList(
    (0): GCNConv(45, 256)
    (1): GCNConv(256, 100)
  )
  (softmax): LogSoftmax(dim=None)
)


In [18]:
def train_with_batch(model, device, data_loader, optimizer, loss_fn):
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
        batch = batch.to(device)
        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            optimizer.zero_grad()
            out=model(batch)
            train_output=out.view([-1,50,2]).to(device)
            train_label = batch.y[:,:,0:2]
            train_availabilities = torch.squeeze(batch.y[:,:,0:1],dim=-1)
            loss=loss_fn(train_label,train_output,train_availabilities)
            loss.backward()
            optimizer.step()

    return loss.item()

def train(model, value_dic, train_idx, optimizer, loss_fn):
    model.train()
    loss = 0
    optimizer.zero_grad()

    out=model(torch.tensor(value_dic["x"]),edge_index = torch.tensor(value_dic["edge_index"],dtype=torch.long), edge_attr = torch.tensor(value_dic["edge_attr"]))
    if len(out.shape) <= 2:
        out = torch.unsqueeze(out,0)

    train_output=out.view([-1,50,2]).to(args['device'])  # 这里暂时是全部训练
    train_label= torch.tensor(value_dic["target_positions"], dtype = torch.float).to(args['device'])
    train_availabilities = torch.tensor(value_dic["target_availabilities"],dtype= torch.int).to(args['device'])
    loss=loss_fn(train_label,train_output,train_availabilities) # 只预测一条路
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
@torch.no_grad()
def test_with_batch(model, device, data_loader, loss_fn):

    model.eval()
    y_true = []
    y_pred = []
    loss = []

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
        batch = batch.to(device)
        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            out=model(batch)
            train_output=out.view([-1,50,2]).to(device)
            train_label = batch.y[:,:,0:2]
            train_availabilities = torch.squeeze(batch.y[:,:,0:1],dim=-1)
            loss=loss_fn(train_label,train_output,train_availabilities)
            y_true.append(train_label.detach().cpu())
            y_pred.append(train_output.detach().cpu())
            loss.append(loss.item())



    return y_true,y_pred,loss

In [19]:
from torch_geometric.data import Data
from torch_geometric.loader import  DataLoader
def convert_framearray_to_dataloader(frame_dic_array):
    pyg_data_list = []
    for ele in frame_dic_array:
        pyg_data_list.append(Data(x = ele["x"],edge_index=ele["edge_index"],edge_attr=ele["edge_attr"],y=torch.cat([ele["target_positions"],torch.unsqueeze(ele["target_availabilities"],dim=-1)],dim = -1)))
    return DataLoader(pyg_data_list,batch_size=16)



In [20]:
for epoch in trange(1, 1 + args["epochs"]):
    loss_whole_scene_list = []
    loss_this_epoch = 0
    for scene_index in trange(0,99):
        scene_ego_dataset = train_ego_dataset.get_scene_dataset(scene_index)
        scene_agent_dataset = train_agent_dataset.get_scene_dataset(scene_index)
        frame_dic_array = transform_one_scene_dataset(scene_ego_dataset,scene_agent_dataset,debug=False)
        dataloader = convert_framearray_to_dataloader(frame_dic_array)


        index = 0
        loss_each_frame = train_with_batch(model_gcn,args["device"],dataloader,optimizer,pytorch_neg_multi_log_likelihood_single)
        loss_this_scene = np.mean(np.array(loss_each_frame))
        loss_this_epoch = loss_this_epoch + loss_this_scene
    if epoch % 5 == 0:
        for scene_index in trange(0,99):
            scene_ego_dataset = train_ego_dataset.get_scene_dataset(scene_index)
            scene_agent_dataset = train_agent_dataset.get_scene_dataset(scene_index)
            frame_dic_array = transform_one_scene_dataset(scene_ego_dataset,scene_agent_dataset,debug=False)
            dataloader = convert_framearray_to_dataloader(frame_dic_array)


            index = 0
            loss_each_frame = train_with_batch(model_gcn,args["device"],dataloader,optimizer,pytorch_neg_multi_log_likelihood_single)
            loss_this_scene = np.mean(np.array(loss_each_frame))
            loss_this_epoch = loss_this_epoch + loss_this_scene
    print(loss_this_epoch)

  0%|          | 0/99 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 26.34it/s]

  1%|          | 1/99 [00:02<04:51,  2.97s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 269.73it/s]

  2%|▏         | 2/99 [00:05<04:22,  2.70s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 259.50it/s]

  3%|▎         | 3/99 [00:11<06:26,  4.03s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 266.00it/s]

  4%|▍         | 4/99 [00:15<06:19,  4.00s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 264.39it/s]

  5%|▌         | 5/99 [00:19<06:24,  4.09s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 260.02it/s]

  6%|▌         | 6/99 [00:23<06:34,  4.25s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 261.24it/s]

  7%|▋         | 7/99 [00:29<07:14,  4.72s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 261.70it/s]

  8%|▊         | 8/99 [00:32<06:22,  4.21s/it]

Iteration: 100%|██████████| 16/16 [00:

8327.304849056041



  0%|          | 0/99 [00:00<?, ?it/s]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 259.02it/s]

  1%|          | 1/99 [00:02<03:33,  2.18s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 259.56it/s]

  2%|▏         | 2/99 [00:04<03:42,  2.29s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 250.55it/s]

  3%|▎         | 3/99 [00:10<06:14,  3.90s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 255.78it/s]

  4%|▍         | 4/99 [00:14<06:19,  3.99s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 257.48it/s]

  5%|▌         | 5/99 [00:18<06:30,  4.16s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 288.35it/s]

  6%|▌         | 6/99 [00:23<06:46,  4.37s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 252.75it/s]

  7%|▋         | 7/99 [00:29<07:29,  4.89s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 292.36it/s]

  8%|▊         | 8/99 [00:32<06:36,  4.36s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 292.29it/s]

  9%|▉         | 9/99 [00:

5167.342105618359



  0%|          | 0/99 [00:00<?, ?it/s]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 264.17it/s]

  1%|          | 1/99 [00:02<03:32,  2.17s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 258.67it/s]

  2%|▏         | 2/99 [00:04<03:41,  2.28s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 255.29it/s]

  3%|▎         | 3/99 [00:10<06:14,  3.90s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 258.81it/s]

  4%|▍         | 4/99 [00:14<06:17,  3.98s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 256.92it/s]

  5%|▌         | 5/99 [00:18<06:30,  4.15s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 290.23it/s]

  6%|▌         | 6/99 [00:23<06:46,  4.37s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 254.18it/s]

  7%|▋         | 7/99 [00:29<07:29,  4.88s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 257.09it/s]

  8%|▊         | 8/99 [00:32<06:36,  4.36s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 260.30it/s]

  9%|▉         | 9/99 [00:

...

4477.16598377208



  0%|          | 0/99 [00:00<?, ?it/s]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 261.09it/s]

  1%|          | 1/99 [00:02<03:28,  2.13s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 256.70it/s]

  2%|▏         | 2/99 [00:04<03:38,  2.26s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 259.13it/s]

  3%|▎         | 3/99 [00:10<06:11,  3.87s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 289.04it/s]

  4%|▍         | 4/99 [00:14<06:13,  3.94s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 253.58it/s]

  5%|▌         | 5/99 [00:18<06:27,  4.12s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 257.14it/s]

  6%|▌         | 6/99 [00:23<06:42,  4.33s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 280.07it/s]

  7%|▋         | 7/99 [00:29<07:28,  4.87s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 294.22it/s]

  8%|▊         | 8/99 [00:32<06:34,  4.34s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 261.17it/s]

  9%|▉         | 9/99 [00:

4492.049984438925



  0%|          | 0/99 [00:00<?, ?it/s]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 261.48it/s]

  1%|          | 1/99 [00:02<03:30,  2.15s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 293.59it/s]

  2%|▏         | 2/99 [00:04<03:38,  2.25s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 251.66it/s]

  3%|▎         | 3/99 [00:10<06:12,  3.88s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 252.56it/s]

  4%|▍         | 4/99 [00:14<06:16,  3.97s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 256.59it/s]

  5%|▌         | 5/99 [00:18<06:29,  4.15s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 244.34it/s]

  6%|▌         | 6/99 [00:23<06:46,  4.38s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 254.96it/s]

  7%|▋         | 7/99 [00:29<07:29,  4.89s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 258.60it/s]

  8%|▊         | 8/99 [00:32<06:38,  4.38s/it]

Iteration: 100%|██████████| 16/16 [00:00<00:00, 256.01it/s]

  9%|▉         | 9/99 [00:

4280.09607644163


In [21]:
print(frame_dic_array[11])

{'x': tensor([[ 1.2106e+01,  0.0000e+00,  0.0000e+00, -1.2104e+00,  2.3242e-02,
         -2.4209e+00,  9.5554e-03, -3.6172e+00,  1.8141e-02, -4.8182e+00,
          2.4359e-02, -6.0246e+00,  1.0994e-02, -7.2214e+00,  4.2989e-02,
         -8.4236e+00,  3.4448e-02, -9.5770e+00,  8.8973e-02, -1.0776e+01,
          8.7089e-02, -1.1978e+01,  9.5563e-02,  0.0000e+00, -1.4057e-03,
         -2.7983e-03, -4.1873e-03, -5.0845e-03, -5.1036e-03, -4.5540e-03,
         -3.5999e-03, -1.9913e-03, -1.0104e-03, -1.3113e-04,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
          1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 5.7551e+00,  0.0000e+00, -2.8422e-14, -5.7545e-01, -8.3046e-03,
         -1.0980e+00,  5.2986e-03, -1.6504e+00,  1.0239e-02, -2.1804e+00,
          8.5397e-03, -2.7181e+00,  4.6928e-02, -3.1525e+00,  7.1776e-02,
         -3.6101e+00,  3.5964e-02, -4.0440e+00,  5.0399e-02, -4.4261e+00,
          1.7812e-01, -5.0083e+

You can obtrain training history results really easily by just accessing `LogReport` class, which is useful for managing a lot of experiments during kaggle competitions.

The history log and model's weight are saved by "extensions" (`LogReport` and `E.snapshot_object` respectively) easily, which is a benefit of using training abstration.

In [22]:
# Let's see training results directory

!ls results/multi_train

cfg.yaml  flags.yaml  log  log.csv  predictor.pt


# Items to try

This kernel shows demonstration run of the training (`debug=True`). You can try these things to see how the score changes at first
 - set debug=False to train with actual training dataset
 - change training hyperparameters (training epoch, change optimizer, scheduler learning rate etc...)
   - Especially, just training much longer time may improve the score.
 
To go further, these items may be nice to try:
 - Change the cnn model (now simple resnet18 is used as baseline modeling)
 - Training the model using full dataset: [lyft-full-training-set](https://www.kaggle.com/philculliton/lyft-full-training-set)
 - Write your own rasterizer to prepare input image as motivation explained in previous kernel.
 - Consider much better scheme to predict multi-trajectory
    - The model just predicts multiple trajectory at the same time in this kernel, but it is possible to collapse "trivial" solution where all trajectory converges to same. How to avoid this?

# Next to go

[Update 2020/9/6]<br/>
Published prediction kernel: [Lyft: Prediction with multi-mode confidence](https://www.kaggle.com/corochann/lyft-prediction-with-multi-mode-confidence)<br/>
Try yourself how good score you can get using only single model without ensemble! :)

To understand the competition in more detail, please refer my other kernels too.
 - [Lyft: Comprehensive guide to start competition](https://www.kaggle.com/corochann/lyft-comprehensive-guide-to-start-competition)
 - [Lyft: Deep into the l5kit library](https://www.kaggle.com/corochann/lyft-deep-into-the-l5kit-library)
 - [Save your time, submit without kernel inference](https://www.kaggle.com/corochann/save-your-time-submit-without-kernel-inference)
 - [Lyft: pytorch implementation of evaluation metric](https://www.kaggle.com/corochann/lyft-pytorch-implementation-of-evaluation-metric)

# Further reference

 - Paper of this Lyft Level 5 prediction dataset: [One Thousand and One Hours: Self-driving Motion Prediction Dataset](https://arxiv.org/abs/2006.14480)
 - [jpbremer/lyft-scene-visualisations](https://www.kaggle.com/jpbremer/lyft-scene-visualisations)

<h3 style="color:red">If this kernel helps you, please upvote to keep me motivated :)<br>Thanks!</h3>